In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
from PIL import Image
import os
import tensorflow as tf
import scipy.signal as signal
import cv2

In [2]:
from PIL import Image
spec_data=[]

In [3]:
def data(root_directory,final_class_labels):
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder
        for filename in filenames:
            if filename.lower().endswith(('.png')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                img=Image.open(audio_path).convert('RGB')
                img=img.resize((128,128))
                img_array = np.asarray(img)
                img_array = img_array / 255
                spec_data.append([img_array,final_class_labels])   
#                 spec_data.append([img_array.tolist(),final_class_labels])   

In [4]:
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_artifact','artifact')
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_artifact','artifact')

In [5]:
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_extrahls','extrahls')
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_extrahls','extrahls')

In [6]:
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_murmur','murmur')
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_murmur','murmur')

In [7]:
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_normal','normal')
data('/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_normal','normal')

In [8]:
import pandas as pd
spec_data=pd.DataFrame(spec_data,columns=['feature','class'])
spec_data.head()

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact


In [9]:
spec_data['class'].value_counts()

class
artifact    720
normal      408
murmur      396
extrahls    222
Name: count, dtype: int64

In [20]:
X=np.array(spec_data['feature'].tolist())
y=np.array(spec_data['class'].tolist())

In [21]:
X.shape

(1746, 128, 128, 3)

In [22]:
#downsampling artifact in dataset-A (drop 50%)

import numpy as np

# Identify the class label you want to downsample
class_label = 'artifact'  # Update with your desired class label

# Separate the data into eliminated and remaining subsets
eliminated_subset_X = X[y == class_label]
eliminated_subset_y = y[y == class_label]
remaining_subset_X = X[y != class_label]
remaining_subset_y = y[y != class_label]

# Randomly select 50% of the data from the remaining subset
num_eliminated_samples = len(eliminated_subset_X)
num_samples_to_select = int(num_eliminated_samples * 0.5)

random_indices = np.random.choice(num_eliminated_samples, size=num_samples_to_select, replace=False)
randomly_selected_X = eliminated_subset_X[random_indices]
randomly_selected_y = eliminated_subset_y[random_indices]

# Combine the eliminated subset and the randomly selected subset
downsampled_X = np.concatenate((remaining_subset_X, randomly_selected_X), axis=0)
downsampled_y = np.concatenate((remaining_subset_y, randomly_selected_y), axis=0)

In [23]:
X=downsampled_X
y=np.array(pd.get_dummies(downsampled_y))

In [24]:
X=np.concatenate((X,X),axis=0)
y=np.concatenate((y,y),axis=0)

In [25]:
X.shape

(2772, 128, 128, 3)

In [26]:
np.unique(y)

array([False,  True])

In [28]:
y

array([[False,  True, False, False],
       [False,  True, False, False],
       [False,  True, False, False],
       ...,
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False]])

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [30]:
X_train.shape

(2217, 128, 128, 3)

In [31]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [32]:
X_train.shape

(1884, 128, 128, 3)

In [33]:
X_valid.shape

(333, 128, 128, 3)

In [34]:
X_test.shape

(555, 128, 128, 3)

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D,MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [36]:
from tensorflow import keras
base_model=keras.applications.MobileNetV2(
    input_shape=(128,128,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model_1 = keras.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])

model_1.summary()

Metal device set to: Apple M1
9406464/9406464 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
 dense (Dense)               (None, 64)                1310784   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 3,569,028
Train

In [37]:
X_train.shape

(1884, 128, 128, 3)

In [38]:
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
checkpoint = ModelCheckpoint('heart_spect_mobilenet_aug_actual.hdf5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
history = model_1.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_valid,y_valid), callbacks=[checkpoint])

Epoch 1/40


2023-07-12 11:31:05.723846: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


59/59 [==============================] - 13s 144ms/step - loss: 1.2792 - accuracy: 0.6555 - val_loss: 0.4548 - val_accuracy: 0.8138
Epoch 2/40
59/59 [==============================] - 6s 98ms/step - loss: 0.4738 - accuracy: 0.8148 - val_loss: 0.3353 - val_accuracy: 0.8709
Epoch 3/40
59/59 [==============================] - 6s 104ms/step - loss: 0.3938 - accuracy: 0.8450 - val_loss: 0.2887 - val_accuracy: 0.9039
Epoch 4/40
59/59 [==============================] - 6s 108ms/step - loss: 0.2908 - accuracy: 0.8880 - val_loss: 0.2209 - val_accuracy: 0.9129
Epoch 5/40
59/59 [==============================] - 6s 100ms/step - loss: 0.2563 - accuracy: 0.8917 - val_loss: 0.1933 - val_accuracy: 0.9159
Epoch 6/40
59/59 [==============================] - 6s 107ms/step - loss: 0.2264 - accuracy: 0.9050 - val_loss: 0.2110 - val_accuracy: 0.9069
Epoch 7/40
59/59 [==============================] - 6s 104ms/step - loss: 0.2005 - accuracy: 0.9130 - val_loss: 0.2146 - val_accuracy: 0.9069
Epoch 8/40
59/59 

In [40]:
test_accuracy=model_1.evaluate(X_test,y_test)
print(test_accuracy[1])

18/18 [==============================] - 2s 131ms/step - loss: 0.0899 - accuracy: 0.9712
0.9711712002754211


In [42]:
y_pred=model_1.predict(X_test)

18/18 [==============================] - 2s 78ms/step


In [43]:
c_acc = tf.keras.metrics.CategoricalAccuracy()
c_acc.update_state(y_test,y_pred)
c_acc.result().numpy()

0.9711712

In [44]:
auc = tf.keras.metrics.AUC()
auc.update_state(y_test,y_pred)
auc.result().numpy()

0.9965928

In [45]:
recall = tf.keras.metrics.Recall()
recall.update_state(y_test,y_pred)
recall.result().numpy()

0.9711712

In [46]:
precision = tf.keras.metrics.Precision()
precision.update_state(y_test,y_pred)
precision.result().numpy()

0.9711712